<h1><center>Coursera Capstone</center></h1>

Welcome to my Capstone Project. The purpose of this is to map generate a visual map of Toronto in Canada. Then we will proceed to a neighborhood comparison.

## Part III: Data Mapping

<p>This section is about data visual mapping. The dataset previously generated will serve as a baseline for venues exploration in the Foursquare API. The information collected about each location will then be categorized and displayed on a city map.</p>
<p><strong>Nota: </strong>for the sake of time saving, the data collected has not been properly cleaned. This notebook is mostly intended to be a proof of concept. We focus here on data mapping implementation rather than results. As a consequence, the K-Mean algorithm results in poor clusters, which means that our error rate is most likely to be high. Nonetheless, as soon as we get a mapping of our datapoints, we can assume that deeper efforts put on data cleaning and processing will output better clusters and a lower error rate.</p> 
</br>
<p><strong>Nota: </strong>you will need your own Foursquare credentials to query the API. To simply check the results you can alternatively load my intermediate results datasets, namely:</p>
<ul>
    <li>toronto_pc_conso.csv : List of Toronto Neighborhoods by Borough and Postal Code</li>
    <li>toronto_pc_agg.csv : List of Toronto Neighborhoods aggregated by Borough and Postal Code</li>
    <li>toronto_venues.csv : List of Toronto Venues for each Postal Code</li>
</ul>
</br>
<p>Some listed Postal Codes did not return any venue on Foursquare's API. In this case it has been considered missing values</p> 

In [1]:
import json

with open("creds.json","r") as file:
    f = json.load(file)
    client_id = f.get("client_id")
    client_secret = f.get("client_secret")
    version = f.get("version")

In [2]:
import pandas as pd

df = pd.read_csv('./toronto_pc_agg.csv')

upc = df['PostalCode'].unique()

In [3]:
df[df['PostalCode'] == 'M7R']

,Unnamed: 0,PostalCode,Borough,Neighborhood,Latitude,Longitude
76,76,M7R,Mississauga,Canada Post Gateway Processing Centre,NaN,NaN


In [4]:
pcdata = df[['PostalCode','Borough','Neighborhood','Latitude','Longitude']]
print(type(pcdata))
pcdata.shape

<class 'pandas.core.frame.DataFrame'>


(103, 5)

In [5]:
pcdata = pcdata.drop(76,axis=0)
print(pcdata[pcdata['PostalCode'] == 'M7R'])
print(pcdata.shape)
pcdata

Empty DataFrame
Columns: [PostalCode, Borough, Neighborhood, Latitude, Longitude]
Index: []
(102, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.6555,-79.3626
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.7223,-79.4504
4,M7A,Queen's Park,Queen's Park,43.6641,-79.3889
5,M9A,Etobicoke,Islington Avenue,43.6662,-79.5282
6,M1B,Scarborough,"Rouge,Malvern",43.8113,-79.1930
7,M3B,North York,Don Mills North,43.7450,-79.3590
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.7063,-79.3094
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.6572,-79.3783


In [7]:
pcdata = pcdata.reset_index(drop=True)
blat = []
blng = []
bname = []
for i in range(0,102):
    borough_name = pcdata.loc[i, 'Borough'] 
    bname.append(borough_name)
    borough_latitude = pcdata.loc[i, 'Latitude'] 
    blat.append(borough_latitude)
    borough_longitude = pcdata.loc[i, 'Longitude'] 
    blng.append(borough_longitude)

### Querying the Foursquare's API 

The query response is formatted in JSON. Thus each response comes as a Python Dictionary from which we must extract the relevant data, namely:
1. Venue name
1. Venue geocoding
1. Venue category

In [8]:
import requests 

limit = 30
radius = 500
resps = []

for i in range(0,102):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        client_id, 
        client_secret, 
        version,
        blat[i],
        blng[i],
        radius,
        limit
    )
    response = requests.get(url).json()
    resps.append(response)

In [9]:
queries = []
for q in resps:
    queries.append(q['response'])

len(queries)

102

In [10]:
print(queries[0])

{'headerLocation': 'Parkwoods - Donalda', 'headerFullLocation': 'Parkwoods - Donalda, Toronto', 'headerLocationGranularity': 'neighborhood', 'totalResults': 4, 'suggestedBounds': {'ne': {'lat': 43.7590000045, 'lng': -79.32378161085641}, 'sw': {'lat': 43.7499999955, 'lng': -79.33621838914358}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b', 'name': 'Brookbanks Park', 'location': {'address': 'Toronto', 'lat': 43.751976046055574, 'lng': -79.33214044722958, 'labeledLatLngs': [{'label': 'display', 'lat': 43.751976046055574, 'lng': -79.33214044722958}], 'distance': 329, 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['Toronto', 'Toronto ON', 'Canada']}, 'categories': [{'id': '4bf58dd8d48988d163941735', 'name': 'Park', 'pluralName': 'Parks', 'shortNam

In [11]:
for i in range(0,102):
    if(len(queries[i])<1):
        lq = len(queries[i])
        print('id {}: {}'.format(i,lq))

In [12]:
grps = []
for item in queries:
    grp = item.get("groups")
    grps.append(grp)
len(grps)

102

In [13]:
s = 0
for i in range(0,102):
    if(grps[i] == None):
        print('id {}: None'.format(i))
    else:
        if(len(grps[i])< 1):
            lg = len(grps[i])
            print('id {}: {}'.format(i,lg))
        else:
            s+=1
print(s)

102


In [14]:
itm = []
for g in grps:
        items = g[0].get("items")
        itm.append(items)

len(itm)

102

In [15]:
itm[0]

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
   'name': 'Brookbanks Park',
   'location': {'address': 'Toronto',
    'lat': 43.751976046055574,
    'lng': -79.33214044722958,
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.751976046055574,
      'lng': -79.33214044722958}],
    'distance': 329,
    'cc': 'CA',
    'city': 'Toronto',
    'state': 'ON',
    'country': 'Canada',
    'formattedAddress': ['Toronto', 'Toronto ON', 'Canada']},
   'categories': [{'id': '4bf58dd8d48988d163941735',
     'name': 'Park',
     'pluralName': 'Parks',
     'shortName': 'Park',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'groups': []}},
  'referralId': 'e-0-4e8d9dcdd5fbbbb6b3003c7b-0'},
 {'reasons': {'count': 0,
   'items': [{'

<p>The number of venues being much bigger than the neighborhoods count, we must populate a new column with as much identifiers as the number of venues returned.</p>
</br>
<p>I have tried to query venues nearby Toronto neighborhoods, with poor results, so I decided to focus on Postal Codes instead.</p>
</br>
<p>A common issue I had to face when extracting information from the response was a Nonetype error, resulting from the fact that some query args e.g Postal Codes returned empty objects {}. This means Foursquare has not any data related to the corresponding entry. To cope with this issue, the initial dataset had to be adapted by removing the corresponding entries prior to populating the new column.</p>

In [16]:
upc2 = upc.tolist()
upc2[76]
upc2.pop(76)

'M7R'

In [17]:
codes = []
venues = []
for i in itm:
    idx = itm.index(i)
    for a in range(0,len(i)):
        codes.append(upc2[idx])
        v = i[a].get("venue")
        venues.append(v)
    
print(len(venues))
print(len(codes))

1305
1305


In [18]:
print(venues[0])

{'id': '4e8d9dcdd5fbbbb6b3003c7b', 'name': 'Brookbanks Park', 'location': {'address': 'Toronto', 'lat': 43.751976046055574, 'lng': -79.33214044722958, 'labeledLatLngs': [{'label': 'display', 'lat': 43.751976046055574, 'lng': -79.33214044722958}], 'distance': 329, 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['Toronto', 'Toronto ON', 'Canada']}, 'categories': [{'id': '4bf58dd8d48988d163941735', 'name': 'Park', 'pluralName': 'Parks', 'shortName': 'Park', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}


In [19]:
places = []
cat = []
lat = []
lng = []
city = []
for i in range(0,1305):
    places.append(venues[i].get("name"))
    cat.append(venues[i].get("categories"))
    city.append(venues[i].get("location").get("city"))
    lat.append(venues[i].get("location").get("lat"))
    lng.append(venues[i].get("location").get("lng"))

In [20]:
print('postcode: {}, name: {}, cat: {}'.format(len(codes),len(places),len(cat)))
print('cities: {}, lat: {}, long: {}'.format(len(city),len(lat),len(lng)))

postcode: 1305, name: 1305, cat: 1305
cities: 1305, lat: 1305, long: 1305


In [21]:
s = 0
for i in range(0,1305):
    if(len(cat[i])< 1):
        s+=1
        
print(s)
len(cat)

0


1305

In [22]:
arr = []
for x in cat:
    n = x[0].get('name')
    arr.append(n)
    
cats = pd.Series(arr)
cats

0                            Park
1            Fast Food Restaurant
2                            Pool
3               Food & Drink Shop
4                    Hockey Arena
5                     Coffee Shop
6           Portuguese Restaurant
7                    Intersection
8                     Pizza Place
9                            Park
10                    Coffee Shop
11                         Bakery
12                 Breakfast Spot
13             Mexican Restaurant
14                 Breakfast Spot
15                    Yoga Studio
16                            Spa
17                    Coffee Shop
18                    Event Space
19                Thai Restaurant
20                            Pub
21                    Coffee Shop
22                     Restaurant
23             Italian Restaurant
24           Gym / Fitness Center
25                    Coffee Shop
26                        Theater
27                     Food Truck
28              Electronics Store
29            

In [23]:
a = pd.Series(codes)
b = pd.Series(places)
c = pd.Series(cats)
d = pd.Series(city)
e = pd.Series(lat)
f = pd.Series(lng)

In [24]:
a = a.reset_index(drop=True)
b = b.reset_index(drop=True)
c = c.reset_index(drop=True)
d = d.reset_index(drop=True)
e = e.reset_index(drop=True)
f = f.reset_index(drop=True)

In [25]:
dataset = pd.concat([a,b,c,d,e,f],axis=1)
dataset.columns = ['PostalCode','Venue','Category','City','Latitude','Longitude']
dataset

,PostalCode,Venue,Category,City,Latitude,Longitude
0,M3A,Brookbanks Park,Park,Toronto,43.751976,-79.332140
1,M3A,KFC,Fast Food Restaurant,None,43.754387,-79.333021
2,M3A,GreenWin pool,Pool,Toronto,43.756232,-79.333842
3,M3A,Variety Store,Food & Drink Shop,Toronto,43.751974,-79.333114
4,M4A,Victoria Village Arena,Hockey Arena,None,43.723481,-79.315635
5,M4A,Tim Hortons,Coffee Shop,Toronto,43.725517,-79.313103
6,M4A,Portugril,Portuguese Restaurant,Toronto,43.725819,-79.312785
7,M4A,Eglinton Ave E & Sloane Ave/Bermondsey Rd,Intersection,Toronto,43.726086,-79.313620
8,M4A,Pizza Nova,Pizza Place,Toronto,43.725824,-79.312860
9,M4A,Wigmore Park,Park,Toronto,43.731023,-79.310771


In [26]:
dataset.to_csv('./toronto_venues.csv')
dataset['Category'].value_counts()

Coffee Shop                98
Café                       61
Pizza Place                42
Park                       39
Restaurant                 35
Bakery                     30
Sandwich Place             30
Italian Restaurant         27
Fast Food Restaurant       25
Grocery Store              23
Gym                        21
American Restaurant        21
Japanese Restaurant        21
Pharmacy                   21
Bar                        19
Clothing Store             18
Gastropub                  16
Steakhouse                 16
Liquor Store               15
Pub                        15
Sushi Restaurant           15
Bank                       15
Gym / Fitness Center       14
Deli / Bodega              14
Greek Restaurant           14
Breakfast Spot             13
Indian Restaurant          12
Hotel                      12
Thai Restaurant            12
Chinese Restaurant         11
                           ..
Health Food Store           1
Bus Station                 1
Auto Deale

When the dataset is done, it has to be transformed prior to performing the classification. 
* The categories encoded as string labels are casted into booleans.
* The table is then grouped by Postal Code to aggregate data
* Then each row is summarized by its top N categories, here we took N=25

In [55]:
# one hot encoding
tor_onehot = pd.get_dummies(dataset[['Category']], prefix="", prefix_sep="")

# add postal code column back to dataframe
tor_onehot['City'] = dataset['City']
tor_onehot['PostalCode'] = dataset['PostalCode'] 

# move neighborhood column to the first column
fixed_columns = [tor_onehot.columns[-1]] + [tor_onehot.columns[-2]] + list(tor_onehot.columns[:-2])
tor_onehot = tor_onehot[fixed_columns]

tor_onehot.head()

,PostalCode,City,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M3A,Toronto,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M3A,Toronto,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M3A,Toronto,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M3A,Toronto,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4A,None,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
tor_grouped = tor_onehot.groupby('PostalCode').mean().reset_index()
tor_grouped

,PostalCode,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,M1C,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,M1E,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.033333,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
3,M1G,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
4,M1H,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
5,M1J,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
6,M1K,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
7,M1L,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
8,M1M,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
9,M1N,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000


In [57]:
num_top_venues = 5

for hood in tor_grouped['PostalCode']:
    print("----"+hood+"----")
    temp = tor_grouped[tor_grouped['PostalCode'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M1B----
                     venue  freq
0             Home Service   1.0
1        Accessories Store   0.0
2  New American Restaurant   0.0
3              Men's Store   0.0
4            Metro Station   0.0


----M1C----
               venue  freq
0                Bar   1.0
1  Accessories Store   0.0
2     Massage Studio   0.0
3        Men's Store   0.0
4      Metro Station   0.0


----M1E----
                  venue  freq
0           Pizza Place  0.13
1           Coffee Shop  0.07
2  Fast Food Restaurant  0.07
3         Grocery Store  0.07
4   Fried Chicken Joint  0.03


----M1G----
                     venue  freq
0        Convenience Store  0.33
1         Insurance Office  0.33
2        Korean Restaurant  0.33
3        Accessories Store  0.00
4  New American Restaurant  0.00


----M1H----
                        venue  freq
0  Construction & Landscaping  0.33
1                      Lounge  0.33
2                       Trail  0.33
3           Accessories Store  0.00
4             

                venue  freq
0         Coffee Shop  0.10
1              Bakery  0.07
2                Café  0.07
3          Restaurant  0.07
4  Italian Restaurant  0.07


----M4Y----
            venue  freq
0         Gay Bar  0.07
1  Ice Cream Shop  0.03
2        Tea Room  0.03
3       Bookstore  0.03
4       Gastropub  0.03


----M5A----
            venue  freq
0     Coffee Shop  0.24
1  Breakfast Spot  0.10
2     Yoga Studio  0.05
3     Event Space  0.05
4      Beer Store  0.05


----M5B----
            venue  freq
0            Café  0.10
1     Coffee Shop  0.07
2  Clothing Store  0.07
3         Theater  0.07
4      Comic Shop  0.03


----M5C----
                 venue  freq
0          Coffee Shop  0.13
1            Gastropub  0.10
2                Hotel  0.07
3           Restaurant  0.07
4  Japanese Restaurant  0.07


----M5E----
           venue  freq
0           Café  0.13
1  Deli / Bodega  0.10
2    Coffee Shop  0.10
3            Bar  0.07
4     Steakhouse  0.07


----M5G----
    

               venue  freq
0        Pizza Place  0.09
1               Bank  0.09
2           Bus Line  0.09
3        Supermarket  0.09
4  Mobile Phone Shop  0.09


----M9V----
                  venue  freq
0         Grocery Store  0.18
1  Fast Food Restaurant  0.09
2              Pharmacy  0.09
3           Pizza Place  0.09
4   Fried Chicken Joint  0.09


----M9W----
                      venue  freq
0                     Hotel   0.2
1               Coffee Shop   0.2
2          Swiss Restaurant   0.2
3       Rental Car Location   0.2
4  Mediterranean Restaurant   0.2




In [39]:
postal = []
for i in x.index:
    pcode = dataset.loc[i,'PostalCode']
    postal.append(pcode)

In [58]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [59]:
import numpy as np

num_top_venues = 25

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
pc_venues_sorted = pd.DataFrame(columns=columns)
pc_venues_sorted['PostalCode'] = tor_grouped['PostalCode']

for ind in np.arange(tor_grouped.shape[0]):
    pc_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

pc_venues_sorted.head()

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
0,M1B,Home Service,Yoga Studio,Dessert Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,...,Fish & Chips Shop,Deli / Bodega,Dance Studio,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping,Concert Hall
1,M1C,Bar,Yoga Studio,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,...,Department Store,Deli / Bodega,Dance Studio,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping,Concert Hall
2,M1E,Pizza Place,Fast Food Restaurant,Grocery Store,Coffee Shop,Discount Store,Intersection,Food & Drink Shop,Supermarket,Sports Bar,...,Burger Joint,Salon / Barbershop,Bank,Convenience Store,Thrift / Vintage Store,Greek Restaurant,Video Game Store,Pharmacy,Sandwich Place,Coworking Space
3,M1G,Convenience Store,Korean Restaurant,Insurance Office,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,...,Department Store,Dessert Shop,Fish & Chips Shop,Deli / Bodega,Dance Studio,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Construction & Landscaping
4,M1H,Lounge,Construction & Landscaping,Trail,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,...,Department Store,Dessert Shop,Fish & Chips Shop,Deli / Bodega,Dance Studio,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store


In [60]:
pc_venues_sorted.shape

(99, 26)

Now that we are ready to do the clustering, the dataset is unlabeled by removing the identifier column [PostalCode]

In [63]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

tor_grouped_clustering = tor_grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:] 

array([2, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 3, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 3, 1, 1, 1, 1], dtype=int32)

In [74]:
missing =[]
missing_id = []
for p in df['PostalCode'].unique():
    k  = pc_venues_sorted[pc_venues_sorted['PostalCode'] == p]
    if(len(k) < 1):
        missing.append(p)
        mid = df[df['PostalCode'] == p].index.values
        missing_id.append(mid[0])
print(missing_id)        
print(missing)
print(len(missing))
print(len(kmeans.labels_))

[62, 76, 95, 97]
['M5N', 'M7R', 'M1X', 'M5X']
4
99


In [76]:
df2 = df.drop(missing_id,axis = 0)
df2 = df2.reset_index(drop = True)

<h3>Finally:</h3>


* The dataset is formatted by merging the initial dataset with the clustering dataset on the [PostalCode] feature as a reference.
* Each entrypoint is displayed of the map based on its Lat and Long features
* Each entrypoint is visually identified on the map by its Cluster label color

In [84]:
pc_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df2[['PostalCode','Borough','Neighborhood','Latitude','Longitude']]

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(pc_venues_sorted.set_index('PostalCode'), on='PostalCode')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
0,M3A,North York,Parkwoods,43.7545,-79.3300,0,Pool,Park,Food & Drink Shop,Fast Food Restaurant,...,Deli / Bodega,Department Store,Dance Studio,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping,Concert Hall
1,M4A,North York,Victoria Village,43.7276,-79.3148,1,Intersection,Coffee Shop,Park,Hockey Arena,...,Diner,Dessert Shop,Yoga Studio,Deli / Bodega,Dance Studio,Fast Food Restaurant,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.6555,-79.3626,1,Coffee Shop,Breakfast Spot,Yoga Studio,Pub,...,Gym / Fitness Center,Convenience Store,Ethiopian Restaurant,Comic Shop,Eastern European Restaurant,Concert Hall,Dumpling Restaurant,Construction & Landscaping,Dog Run,Deli / Bodega
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.7223,-79.4504,1,Clothing Store,Men's Store,Sporting Goods Shop,Cosmetics Shop,...,Video Game Store,Coffee Shop,American Restaurant,Leather Goods Store,Shopping Mall,Eastern European Restaurant,Dumpling Restaurant,Event Space,Ethiopian Restaurant,Yoga Studio
4,M7A,Queen's Park,Queen's Park,43.6641,-79.3889,1,Coffee Shop,Japanese Restaurant,Italian Restaurant,Theater,...,Dance Studio,Burrito Place,Gym,Gastropub,Wings Joint,Hobby Shop,Arts & Crafts Store,General Entertainment,Adult Boutique,Creperie


In [85]:
toronto_merged.shape

(99, 31)

In [86]:
from IPython.display import Image 
from IPython.core.display import HTML
!pip install folium --user

import folium 

In [87]:
import matplotlib.cm as cm
import matplotlib.colors as colors

latitude = toronto_merged['Latitude'].mean()
longitude = toronto_merged['Longitude'].mean()
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [88]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
0,North York,0,Pool,Park,Food & Drink Shop,Fast Food Restaurant,Yoga Studio,Dessert Shop,Falafel Restaurant,Event Space,...,Deli / Bodega,Department Store,Dance Studio,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping,Concert Hall
5,Etobicoke,0,Pharmacy,Park,Grocery Store,Skating Rink,Bank,Dessert Shop,Falafel Restaurant,Event Space,...,Deli / Bodega,Department Store,Fast Food Restaurant,Dance Studio,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping
7,North York,0,Pool,Park,Basketball Court,Furniture / Home Store,Yoga Studio,Dessert Shop,Falafel Restaurant,Event Space,...,Deli / Bodega,Department Store,Fast Food Restaurant,Dance Studio,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping
13,North York,0,Home Service,Gym,Park,River,Yoga Studio,Department Store,Falafel Restaurant,Event Space,...,Dessert Shop,Dance Studio,Deli / Bodega,Fast Food Restaurant,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping
16,York,0,Field,Trail,Playground,Hockey Arena,Coffee Shop,Grocery Store,Park,Deli / Bodega,...,Ethiopian Restaurant,Event Space,Department Store,Creperie,Dance Studio,Cuban Restaurant,Farmers Market,Coworking Space,Cosmetics Shop,Convenience Store
21,York,0,Park,Fast Food Restaurant,Sporting Goods Shop,Market,Mexican Restaurant,Beer Store,Bakery,Pharmacy,...,Dog Run,Falafel Restaurant,Ethiopian Restaurant,Yoga Studio,Diner,Department Store,Deli / Bodega,Dance Studio,Cuban Restaurant,Creperie
27,North York,0,Park,Bus Stop,Residential Building (Apartment / Condo),Yoga Studio,Department Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,...,Deli / Bodega,Fast Food Restaurant,Dance Studio,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping,Concert Hall
35,East York,0,Park,Coffee Shop,Convenience Store,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,...,Department Store,Dessert Shop,College Theater,Deli / Bodega,Dance Studio,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Construction & Landscaping
39,North York,0,Flower Shop,Park,Trail,Yoga Studio,Dessert Shop,Farmers Market,Falafel Restaurant,Event Space,...,Deli / Bodega,Department Store,Field,Dance Studio,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping
52,North York,0,Playground,College Stadium,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,...,Deli / Bodega,Dance Studio,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping,Concert Hall,Comic Shop


In [89]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
1,North York,1,Intersection,Coffee Shop,Park,Hockey Arena,Portuguese Restaurant,Pizza Place,Falafel Restaurant,Event Space,...,Diner,Dessert Shop,Yoga Studio,Deli / Bodega,Dance Studio,Fast Food Restaurant,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop
2,Downtown Toronto,1,Coffee Shop,Breakfast Spot,Yoga Studio,Pub,Electronics Store,Spa,Beer Store,Mexican Restaurant,...,Gym / Fitness Center,Convenience Store,Ethiopian Restaurant,Comic Shop,Eastern European Restaurant,Concert Hall,Dumpling Restaurant,Construction & Landscaping,Dog Run,Deli / Bodega
3,North York,1,Clothing Store,Men's Store,Sporting Goods Shop,Cosmetics Shop,Toy / Game Store,Tea Room,Jewelry Store,Bakery,...,Video Game Store,Coffee Shop,American Restaurant,Leather Goods Store,Shopping Mall,Eastern European Restaurant,Dumpling Restaurant,Event Space,Ethiopian Restaurant,Yoga Studio
4,Queen's Park,1,Coffee Shop,Japanese Restaurant,Italian Restaurant,Theater,Liquor Store,Bubble Tea Shop,Café,Mexican Restaurant,...,Dance Studio,Burrito Place,Gym,Gastropub,Wings Joint,Hobby Shop,Arts & Crafts Store,General Entertainment,Adult Boutique,Creperie
8,East York,1,Pizza Place,Fast Food Restaurant,Gastropub,Pet Store,Breakfast Spot,Bank,Intersection,Pharmacy,...,Dog Run,Discount Store,Diner,Dessert Shop,Department Store,Dance Studio,Falafel Restaurant,Cuban Restaurant,Creperie,Coworking Space
9,Downtown Toronto,1,Café,Coffee Shop,Clothing Store,Theater,Burger Joint,Hotel,Japanese Restaurant,Steakhouse,...,Mexican Restaurant,Beer Bar,Comic Shop,Vegetarian / Vegan Restaurant,Art Gallery,American Restaurant,Movie Theater,Burrito Place,Music Venue,Ramen Restaurant
10,North York,1,Italian Restaurant,Rental Car Location,Fast Food Restaurant,Fish Market,Latin American Restaurant,Japanese Restaurant,Mediterranean Restaurant,Pizza Place,...,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store,Dessert Shop,Convenience Store,Department Store,Comic Shop,Falafel Restaurant,Deli / Bodega
11,Etobicoke,1,Pizza Place,Coffee Shop,Bank,Dessert Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,...,Department Store,Fish & Chips Shop,Deli / Bodega,Dance Studio,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping
14,East York,1,Spa,Beer Store,Convenience Store,Video Store,Asian Restaurant,Diner,Fast Food Restaurant,Farmers Market,...,Discount Store,Department Store,Dessert Shop,Fish & Chips Shop,Deli / Bodega,Dance Studio,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop
15,Downtown Toronto,1,Coffee Shop,Gastropub,Hotel,Restaurant,Japanese Restaurant,New American Restaurant,Café,Poke Place,...,Food Truck,Speakeasy,Gym,American Restaurant,Thai Restaurant,Theater,BBQ Joint,Bagel Shop,Dumpling Restaurant,Field


In [90]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
6,Scarborough,2,Home Service,Yoga Studio,Dessert Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,...,Fish & Chips Shop,Deli / Bodega,Dance Studio,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping,Concert Hall


In [92]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
63,York,3,Park,Yoga Studio,Dessert Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,...,Health Food Store,Deli / Bodega,Dance Studio,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping,Concert Hall
68,West Toronto,3,Park,Yoga Studio,Dessert Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,...,Health Food Store,Deli / Bodega,Dance Studio,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping,Concert Hall


In [93]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
12,Scarborough,4,Bar,Yoga Studio,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,...,Department Store,Deli / Bodega,Dance Studio,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store,Construction & Landscaping,Concert Hall


We can then explore each cluster by comparing its features. Although the computational results are quite poor, better results could have been obtained with the proper efforts put on the data collection and cleaning phase. Nonetheless we can assume that the job job is done since we have got results on each step.